In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import os


import cv2
import shutil
import numpy as np
import pandas as pd
import albumentations as A

from PIL import Image, ImageDraw
#from ultralytics import YOLO

from utils.utils import list_file_r, copy_image_to_sub_dir
from utils.data_augmentation import batch_augment_with_bbox, batch_augment_with_mask

In [45]:


augmentor = A.Compose([
    # Transform (random noises)
    A.transforms.Spatter(p=0.2),
    A.transforms.RandomFog(p=0.2),
    A.transforms.RandomShadow(p=0.2),
    A.RandomBrightnessContrast(p=0.2),
    # Blur effect
    A.MotionBlur(p=0.2),
    A.Defocus(),
    # Crop
    #A.RandomCrop(width=450, height=450),
    # Dropout
    #   none
    # Geometric
    A.ShiftScaleRotate(rotate_limit=90, p=0.5),
    #A.HorizontalFlip(p=0.1),
    #A.Rotate(),
    A.FDA(reference_images=[cv2.imread('datasets/CF_simulation/real_image/test/hdt5_0010.tif')],read_fn=lambda x:x,p=1)], 
    bbox_params=A.BboxParams(format='yolo', min_visibility=0.2))

In [46]:
img = cv2.imread('datasets/CF_simulation/images/01_line_Radius4_2_Density1500_1000_Width50_Reflection_0.tif')
bbox = np.loadtxt('datasets/CF_simulation/bbox/01_line_Radius4_2_Density1500_1000_Width50_Reflection_0.txt', delimiter=' ')
bbox = np.concatenate((bbox, bbox[:,0:1]), axis=-1)[:,1:]

In [34]:
temp = np.random.randint(0, 256, [100, 100, 3], dtype=np.uint8)
target_temp = np.random.randint(0, 256, [100, 100, 3], dtype=np.uint8)
aug = A.Compose([A.FDA([target_temp], p=1, read_fn=lambda x: x)])
result = aug(image=temp)

In [54]:
augmented = augmentor(image=img,bboxes=bbox)

In [55]:
Image.fromarray(augmented['image']).show()

In [118]:
np.array(augmented['bboxes'])

array([[0.79821737, 0.61650781, 0.40356525, 0.76698438, 0.        ],
       [0.72620155, 0.5829415 , 0.54155936, 0.834117  , 0.        ],
       [0.58518868, 0.54937519, 0.54155936, 0.90124962, 0.        ],
       [0.4441758 , 0.51580888, 0.54155936, 0.96838223, 0.        ],
       [0.30316292, 0.47466304, 0.54155936, 0.94932607, 0.        ],
       [0.21646486, 0.44109673, 0.43292973, 0.88219346, 0.        ],
       [0.14595842, 0.40753042, 0.29191685, 0.81506084, 0.        ]])

In [119]:
res = np.array(augmented['bboxes'])
res = np.concatenate((res[:,-1:], res), axis=-1)[:,:-1]
res

array([[0.        , 0.79821737, 0.61650781, 0.40356525, 0.76698438],
       [0.        , 0.72620155, 0.5829415 , 0.54155936, 0.834117  ],
       [0.        , 0.58518868, 0.54937519, 0.54155936, 0.90124962],
       [0.        , 0.4441758 , 0.51580888, 0.54155936, 0.96838223],
       [0.        , 0.30316292, 0.47466304, 0.54155936, 0.94932607],
       [0.        , 0.21646486, 0.44109673, 0.43292973, 0.88219346],
       [0.        , 0.14595842, 0.40753042, 0.29191685, 0.81506084]])

In [120]:
#res = pd.DataFrame(res)
#res[:,0] = res[:,0].astype(int)
np.savetxt('test.txt',res, fmt=' '.join(['%i']+['%1.4f']*4))

In [ ]:
augmentor = A.Compose([
    # Transform (random noises)
    A.transforms.Spatter(p=0.2),
    A.transforms.RandomFog(p=0.2, fog_coef_lower=0.1, fog_coef_upper=0.4),
    A.transforms.RandomShadow(p=0.2),
    A.RandomBrightnessContrast(p=0.2),
    # Blur effect
    A.MotionBlur(p=0.2),
    A.Defocus(p=0.2),
    # Crop
    A.BBoxSafeRandomCrop(p=0.5),
    # Dropout
    #   none
    # Geometric
    A.ShiftScaleRotate(rotate_limit=45, p=0.75)
    #A.HorizontalFlip(p=0.1),
    #A.Rotate(),
], bbox_params=A.BboxParams(format='yolo', min_visibility=0.2))

#batch_augment_with_bbox(images_path='datasets/CF_simulation/images/', bbox_path='datasets/CF_simulation/bbox', augmentor=augmentor, random_choice=1.0)


In [81]:
transforms = [
    # Blur effect
    A.MotionBlur(p=0.2, blur_limit=5),
    A.Defocus(p=0.2, alias_blur=0.3),
    # Crop
    #A.BBoxSafeRandomCrop(p=0.5),
    A.RandomCrop(640, 640, p=0.5),
    # Dropout
    #   none
    # Geometric
    A.ShiftScaleRotate(rotate_limit=45, p=0.75),
    #A.HorizontalFlip(p=0.1),
    #A.Rotate(),
    # Transform (random noises),
    A.transforms.Spatter(p=0.2),
    A.transforms.RandomFog(p=0.2, fog_coef_lower=0.02, fog_coef_upper=0.1),
    A.transforms.RandomShadow(p=0.2),
    A.RandomBrightnessContrast(p=0.2),
]

batch_augment_with_mask(images_path='datasets/CF_simulation/images/', mask_path='datasets/CF_simulation/mask/', transforms=transforms, FDA_targets='real_image/FDA_targets', random_choice=1.0)